# Assigment 5

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

In this path **..\_data\sbs\B_RawData\bancos** you will find scraped data from [this link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B). We get all the information of the last available day of every each month.

1. Generate a function called **import_data_sbs_banks** that take as arguments `start_month`, `start_year`, `end_month` and `end_year`. This function should be able to import the files in **..\_data\sbs\B_RawData\bancos** taking into consideration these arguments. Additionally, it should work if it is only given the first two arguments `(start_mont, start_year)`. The function should `returns` a nested dictionary. The `main key` must be year and `second key` must be `month`. The values should be the dataframes.

In [11]:
# Install the packages as required.
import os
import pandas as pd
from datetime import datetime

In [12]:
def import_data_sbs_banks(start_month, start_year, end_month = None, end_year = None):

    # Retrieve all the files located in the specified path and store them in a list
    path = '_data/sbs/B_RawData/bancos'
    file_list = [os.path.join(path, file) for file in os.listdir(path)]
    
    # Create empty dictionary to save dataframes
    excel_dict = {} 
    
    # Iterate over each element in "file_list" 
    for file in file_list:
        # Read excel file and save in a provisional df
        df_tmp = pd.read_excel(file)
        # Create column "date_info" with the day information
        df_tmp["date_info"] = os.path.splitext(file)[0][45:]
        # Change the type of the "date_info" column to date 
        df_tmp["date_info"] = pd.to_datetime(df_tmp["date_info"], format = "%d_%m_%Y")
        # Save all files in the dictionary 
        excel_dict[file] = df_tmp     
    
    # In case we only define the values of start_month and start_year
    if end_month is None:
        end_month = start_month
    if end_year is None:
        end_year = start_year    
        
    # Create start_date and end_date by values of arguments
    # Use "pd.to_datetime" to transform string to date
    start_date = pd.to_datetime(f'{start_month}/01/{start_year}', format = '%m/%d/%Y')
    # Specify "end_month+1" in end_date so that the range covers the whole month
    # Add a condition for December, in that case, it is necessary to take as reference the first day of next year
    if end_month == 12:
        end_date = pd.to_datetime(f'01/01/{end_year + 1}', format = '%m/%d/%Y')
    else: 
        end_date = pd.to_datetime(f'{end_month + 1}/01/{end_year}', format = '%m/%d/%Y') 

    # Select dfs whose "date_info" column falls between start_date and end_date
    selected_dfs = {
        k: v for k, v in excel_dict.items() 
        if start_date <= pd.Timestamp(v['date_info'].iloc[0]) <= end_date
    }
    
    # Sort the values in selected_dfs by date
    sorted_dfs = sorted(selected_dfs.items(), key =lambda x: (x[1]['date_info'].dt.year.values[0],
                                                              x[1]['date_info'].dt.month.values[0]))
    
    # Create empty dictionary to store nested dataframes
    nested_dict = {}
    
    # Nest selected dataframes in the dictionary
    # Use the years as the first level keys and the months as the second level keys
    # The final dict will have the next structure: "nested_dict[year][month] = value"
    for key, value in sorted_dfs:
        year  = value['date_info'].dt.year.values[0]
        month = value['date_info'].dt.month.values[0]
        nested_dict.setdefault(year, {})[month] = value

    return nested_dict

In [13]:
import_data_sbs_banks(1, 2021,12,2021)

ValueError: time data "004" doesn't match format "%d_%m_%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.